# TDA Analysis

**Goal:** Compute persistent homology for clean CIFAR-10 features and visualize topological structure

### Overview
In this notebook, we will:
1. Load the clean features (50-dim PCA-reduced)
2. Compute persistence diagrams for H0 (connected components) and H1 (loops)
3. Visualize persistence diagrams
4. Analyze Betti curves
5. Extract topological statistics
6. Save diagrams for later comparison with adversarial features

---

### Why TDA?
Traditional ML looks at distances and gradients in feature space. TDA reveals the shape of the data:
- **H0 (homology dim 0):** Connected components: how clustered is the data?
- **H1 (homology dim 1):** Loops/cycles: are there circular structures?

**Our hypothesis:** Adversarial perturbations will fragment clusters (more H0 features) and **destroy** loops (fewer H1 features).